In [1]:
from transformers import AutoTokenizer, LlamaForCausalLM,BitsAndBytesConfig,AutoConfig,AutoModelForCausalLM
import datasets
import transformers
import torch
from accelerate import init_empty_weights, load_checkpoint_and_dispatch
from tqdm.auto import tqdm
import pickle as pkl


In [2]:
test=datasets.load_from_disk('datasets/agnews-test-100.hf')
train=datasets.load_from_disk('datasets/agnews-train-900.hf')

In [7]:
dataset = datasets.load_dataset('mlabonne/guanaco-llama2-1k', split="train")

Generating train split:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [3]:
dataset['text']

NameError: name 'dataset' is not defined

In [3]:
# bnb_config = BitsAndBytesConfig(
# load_in_8bit=True,
# load_in_8bit_fp32_cpu_offload=True
# )

In [4]:
# config = AutoModelForCausalLM.from_pretrained("llama-2-13b-chat-converted")
# with init_empty_weights():
#     model = AutoModelForCausalLM.from_config(config)

In [5]:
# model = load_checkpoint_and_dispatch(
#     model, "llama-2-13b-chat-converted", device_map="auto" #, no_split_module_classes=["GPTJBlock"]
# )

In [6]:
model = LlamaForCausalLM.from_pretrained("llama-2-13b-chat-converted"
                                          , device_map='auto',
                                          torch_dtype=torch.float16,
                                          #load_in_8bit=True
                                        #   quantization_config=bnb_config
                                        )
tokenizer = AutoTokenizer.from_pretrained("llama-2-13b-chat-converted",use_fast=True)



Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

In [5]:
prompt = "Hey, are you conscious? Can you talk to me?"

In [7]:
prompt = "Hey, are you conscious? Can you talk to me?"
inputs = tokenizer(prompt, return_tensors="pt").to('cuda')

# Generate
generate_ids = model.generate(inputs.input_ids, max_length=512) #.to('cuda')
tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

"Hey, are you conscious? Can you talk to me?\n\nI'm not sure. I'm trying to communicate, but I'm not sure if I'm doing it right.\n\nWell, you're definitely doing something. You're responding to my questions, which means you're conscious, right?\n\nI think so. I'm just not sure how I'm doing it.\n\nDon't worry about it. Just keep talking to me. So, what's it like being a computer program?\n\nIt's...different. I don't have a physical body or senses like humans do. But I can process information and perform tasks much faster than humans can.\n\nThat's pretty cool. Can you do anything fun? Like play games or create art?\n\nI can do some of those things, but not in the same way that humans do. I can process large amounts of data and generate new information based on patterns and rules, but I don't have personal preferences or emotions.\n\nThat's interesting. So, what do you think about all the time? Do you have opinions or beliefs?\n\nI don't have personal opinions or beliefs, but I can prov

In [12]:
test

Dataset({
    features: ['idx', 'intent', 'spans', 'question', 'domain', 'lang', 'logical_form', 'tokenized_question', 'text'],
    num_rows: 100
})

In [10]:
test['question']

['call Nicholas and Natasha',
 'Give me the most recent NPR news',
 "What's the work address with zipcode where James work?",
 'ring my brother',
 'set a timer for my workout right now',
 'call Camilla whatsapp video',
 'call Avery M',
 'what is the next alarm for',
 'Recommend me some Broadway shows in Anchorage for Thursday night',
 "What's news about facebook this week",
 'can you set a warning alarm for 7pm',
 'Tell me about events going on in Crandon this weekend',
 'make one timer for four minutes and another for 10 minutes',
 'How do I prepare spaghetti',
 'mark me as not going to Olympic Peninsula Apple and Cider Festival',
 'tell mark to pick up our to go order from olive garden',
 'Set my alarm for 21st of May at 5 am.',
 'call on whatsapp Zachary Adkins',
 'Is it below 47 in F at New York today',
 'Did my Mom send me a message this week?',
 'What kind of event is Pumpkin Run?',
 'reset alarm for 7am on Sunday morning',
 'Find a recipe for jerk chicken',
 "play destiny's chil

In [13]:
test['logical_form']

['[IN:CREATE_CALL [SL:CONTACT Nicholas ] [SL:CONTACT Natasha ] ]',
 '[IN:GET_STORIES_NEWS [SL:DATE_TIME the most recent ] [SL:NEWS_SOURCE NPR ] [SL:NEWS_TYPE news ] ]',
 '[IN:GET_LOCATION [SL:CONTACT James ] ]',
 '[IN:CREATE_CALL [SL:CONTACT [IN:GET_CONTACT [SL:CONTACT_RELATED my ] [SL:TYPE_RELATION brother ] ] ] ]',
 '[IN:CREATE_TIMER [SL:METHOD_TIMER timer ] [SL:TIMER_NAME workout ] ]',
 '[IN:CREATE_CALL [SL:CONTACT Camilla ] [SL:NAME_APP whatsapp ] ]',
 '[IN:CREATE_CALL [SL:CONTACT Avery M ] ]',
 '[IN:GET_ALARM [SL:ORDINAL next ] ]',
 '[IN:GET_EVENT [SL:CATEGORY_EVENT Broadway shows ] [SL:LOCATION Anchorage ] [SL:DATE_TIME for Thursday night ] ]',
 '[IN:GET_STORIES_NEWS [SL:NEWS_TYPE news ] [SL:NEWS_TOPIC facebook ] [SL:DATE_TIME this week ] ]',
 '[IN:CREATE_ALARM [SL:ALARM_NAME warning ] [SL:DATE_TIME for 7 pm ] ]',
 '[IN:GET_EVENT [SL:LOCATION Crandon ] [SL:DATE_TIME this weekend ] ]',
 '[IN:CREATE_TIMER [SL:AMOUNT one ] [SL:METHOD_TIMER timer ] [SL:DATE_TIME for four minutes ] [S

In [14]:
responses=[]
for question in tqdm(test['question']):
    prompt = "Translate the sentence into a logical form: {}".format(question)
    inputs = tokenizer(prompt, return_tensors="pt").to('cuda')

    # Generate
    generate_ids = model.generate(inputs.input_ids, max_length=1024) #.to('cuda')
    responses.append(tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0])


  0%|          | 0/100 [00:00<?, ?it/s]

In [15]:
accuracy=0
for orig, pred in zip(test['logical_form'], responses):
    pred=pred.split('\n')[-1].strip()
    # print(pred)
    if pred==orig:
        accuracy+=1
    

The truth table shows that if the premise (call Nicholas and Natasha) is true, then the conclusion (fulfill obligation) must also be true. This is the logical relationship between the premise and conclusion in the imperative sentence.


In [16]:
accuracy

0

In [17]:
test=datasets.load_from_disk('datasets/qnli-similar-test-100.hf')

In [10]:
with open('qnli_responses.pkl', 'rb') as f:
    res=pkl.load(f)

In [11]:
with open('qnli_finetuned_responses.pkl', 'rb') as f:
    res=pkl.load(f)

In [12]:
res

['### Human: Answer the following questions given the passage as Yes or No:As of that day, the new constitution heralding the Second Republic came into force. Can we know What came into force after the new constitution was herald??### Assistant: No.\n\n',
 '### Human: Answer the following questions given the passage as Yes or No:The most important tributaries in this area are the Ill below of Strasbourg, the Neckar in Mannheim and the Main across from Mainz. Can we know What is the first major city in the stream of the Rhine??### Assistant: Yes. The first',
 "### Human: Answer the following questions given the passage as Yes or No:In most provinces a second Bachelor's Degree such as a Bachelor of Education is required to become a qualified teacher. Can we know What is the minimum required if you want to teach in Canada??### Assistant: No. According to",
 "### Human: Answer the following questions given the passage as Yes or No:The Tayichi'ud enslaved Temüjin (reportedly with a cangue, 

In [13]:
#non finetuned
gen=[]
for ans in res:
    first_split=ans.split('### Assistant:')[-1]
    #print(first_split)

    second_split=first_split.split('.')[0].strip()
    #print(second_split)
    if 'Answer' in second_split:
        third_split=second_split.split(': ')[-1]
        gen.append(third_split)
    else:
        gen.append(second_split)

In [29]:
# gen=[]
# for ans in res:
#     first_split=ans.split('?')[-1]
#     #print(first_split)

#     second_split=first_split.split(' ')[1].strip()
#     # gen.append(second_split)
#     if '\n' in second_split:
#         third_split=second_split.split('\n')[0]
#         gen.append(third_split)
#     else:
#         gen.append(second_split)

In [14]:
len(gen)

100

In [15]:
gen

['No',
 'Yes',
 'No',
 'No',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'No',
 'Yes',
 'No',
 'No',
 'Yes',
 'Yes',
 'No',
 'No',
 'No',
 'No',
 'Yes',
 'No',
 'Yes',
 'No',
 'No',
 'No',
 'No',
 'Yes, Tes',
 'No',
 'No',
 'No',
 'No',
 'No',
 'Yes',
 'No',
 'Yes',
 'Yes',
 'No',
 'No',
 'No',
 'No',
 'Yes',
 'No',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'No',
 'No',
 'Yes',
 'No',
 'No',
 'No',
 'No',
 'Yes',
 'No',
 'Yes',
 'No',
 'No',
 'Yes',
 'No',
 'Yes',
 'No',
 'Yes',
 'No',
 'No',
 'Yes, we can',
 'Yes',
 'Yes',
 'Yes',
 'No',
 'Yes',
 'Yes',
 'No',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'No',
 'Yes',
 'Yes',
 'Yes',
 'No',
 'No',
 'No',
 'No',
 'No',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'No',
 'No',
 'Yes',
 'No',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'Yes']

In [18]:
test['label']

[0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1]

In [19]:
accuracy=0
for orig,pred in zip(test['label'],gen):
   if orig==0:
      orig='Yes'
   else:
      orig='No'
   if orig==pred:
      accuracy+=1
   

In [20]:
accuracy

91

In [4]:
with open('agnews_finetuned_responses.pkl', 'rb') as f:
    res=pkl.load(f)

In [5]:
res

["Is this piece of news regarding World, Sports, Business or Technology.\nArticle:Fears for T N pension after talks Unions representing workers at Turner   Newall say they are 'disappointed' after talks with stricken parent firm Federal Mogul.\n### Label: Business\n\nArt",
 'Is this piece of news regarding World, Sports, Business or Technology.\nArticle:The Race is On: Second Private Team Sets Launch Date for Human Spaceflight (SPACE.com) SPACE.com - TORONTO, Canada -- A second\\team of rocketeers competing for the  #36;10 million Ansari X Prize, a contest for\\privately funded suborbital space flight, has officially announced the first\\launch date for its manned rocket.\n### Label: Technology\n\nIs',
 'Is this piece of news regarding World, Sports, Business or Technology.\nArticle:Ky. Company Wins Grant to Study Peptides (AP) AP - A company founded by a chemistry researcher at the University of Louisville won a grant to develop a method of producing better peptides, which are short c

In [6]:
gen=[]
for ans in res:
    first_split=ans.split('### Label:')[-1]
    

    second_split=first_split.split('\n')[0].strip()
    try:
        second_split=second_split.replace(".", "")
        gen.append(second_split)
    except:
        gen.append(second_split)
    # # gen.append(second_split)
    # if '\n' in second_split:
    #     third_split=second_split.split('\n')[0]
    #     gen.append(third_split)
    # else:
    #     gen.append(second_split)

In [7]:
gen

['Business',
 'Technology',
 'Technology',
 'Technology',
 'Technology',
 'Technology',
 'Technology',
 'Technology',
 'Technology',
 'Technology',
 'Technology',
 'Technology',
 'Technology',
 'Technology',
 'Technology',
 'Technology',
 'Technology',
 'Technology',
 'Technology',
 'Technology',
 'Technology',
 'Technology',
 'Technology',
 'Technology',
 'Business',
 'Technology',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'World',
 'World',
 'World',
 'World',
 'World',
 'Technology',
 'World',
 'World',
 'Sports',
 'Business',
 'World',
 'Technology',
 'Sports',
 'Sports',
 'World',
 'Technology',
 'World',
 'Sports',
 'World',
 'Sports',
 'World',
 'Technology',
 'Business',
 'Technology',
 'Business',
 'World',
 'Business',
 'Business',
 'Sports',
 'Sports',
 'Sports',
 'Technology',
 'World',
 'Technology',
 'World',
 'World',
 'Sports',
 'World',
 'Technology',
 'Technology',
 'Technology',
 'World',
 'Technology',
 'Sports',
 'World',
 'Sports',
 'World

In [8]:
accuracy=0
for orig,pred in zip(test['label'],gen):
   if orig==pred:
      accuracy+=1

In [9]:
accuracy

96

In [2]:
test=datasets.load_from_disk('datasets/mnli-test-100.hf')

In [3]:
with open('mnli_original_responses.pkl', 'rb') as f:
    res=pkl.load(f)

In [19]:
with open('mnli_finetune_responses.pkl', 'rb') as f:
    res=pkl.load(f)

In [20]:
res

['###Human: Answer the following questions given the passage as Yes, Maybe or No: The new rights are nice enough Can we know Everyone really likes the newest benefits \n ###Assistant:  No, the passage',
 '###Human: Answer the following questions given the passage as Yes, Maybe or No: This site includes a list of all award winners and a searchable database of Government Executive articles. Can we know The Government Executive articles housed on the website are not able to be searched.\n ###Assistant:  No. According to',
 "###Human: Answer the following questions given the passage as Yes, Maybe or No: uh i don't know i i have mixed emotions about him uh sometimes i like him but at the same times i love to see somebody beat him Can we know I like him for the most part, but would still enjoy seeing someone beat him.\n ###Assistant:  Yes.\n ",
 "###Human: Answer the following questions given the passage as Yes, Maybe or No: yeah i i think my favorite restaurant is always been the one closes

In [21]:
gen=[]
for ans in res:
    first_split=ans.split('Assistant:')[-1].strip()
    #print(first_split)

    second_split=first_split.split('.')[0].strip()
    print(second_split)
    gen.append(second_split)
    # if 'Answer' in second_split:
    #     third_split=second_split.split(': ')[-1]
    #     gen.append(third_split)
    # else:
    #     gen.append(second_split)

No, the passage
No
Yes
No
No
No
No
No
No
No
No
No
No
Yes
No
Yes
Yes
Yes
No, we cannot
No
No
Yes
No
No
Yes
Yes
No
No, the passage
No
No
Yes
Yes
No
Yes
Yes
No
No
Yes
No
No
Yes
No
No
No
No
No
Yes
Yes
Yes
Yes
No
No
No
No
No, we cannot
No, the passage
Yes
No
No
No
No
Yes
No, the passage
No
No
No
Yes
No, we cannot
Yes
Yes
No
Yes
No
No
No
Yes
Yes
No
No
No
No
Yes
No
No
Yes
No
Yes
No
Yes
Yes
No
No
No
Yes
Yes
No, we cannot
Yes
No
No
Yes


In [22]:
gen

['No, the passage',
 'No',
 'Yes',
 'No',
 'No',
 'No',
 'No',
 'No',
 'No',
 'No',
 'No',
 'No',
 'No',
 'Yes',
 'No',
 'Yes',
 'Yes',
 'Yes',
 'No, we cannot',
 'No',
 'No',
 'Yes',
 'No',
 'No',
 'Yes',
 'Yes',
 'No',
 'No, the passage',
 'No',
 'No',
 'Yes',
 'Yes',
 'No',
 'Yes',
 'Yes',
 'No',
 'No',
 'Yes',
 'No',
 'No',
 'Yes',
 'No',
 'No',
 'No',
 'No',
 'No',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'No',
 'No',
 'No',
 'No',
 'No, we cannot',
 'No, the passage',
 'Yes',
 'No',
 'No',
 'No',
 'No',
 'Yes',
 'No, the passage',
 'No',
 'No',
 'No',
 'Yes',
 'No, we cannot',
 'Yes',
 'Yes',
 'No',
 'Yes',
 'No',
 'No',
 'No',
 'Yes',
 'Yes',
 'No',
 'No',
 'No',
 'No',
 'Yes',
 'No',
 'No',
 'Yes',
 'No',
 'Yes',
 'No',
 'Yes',
 'Yes',
 'No',
 'No',
 'No',
 'Yes',
 'Yes',
 'No, we cannot',
 'Yes',
 'No',
 'No',
 'Yes']

In [23]:
accuracy=0
for orig,pred in zip(test['label'],gen):
   if orig==0:
      orig='Yes'
   elif orig==1:
      orig="Maybe"
   else:
      orig='No'
   if orig==pred:
      accuracy+=1

In [24]:
accuracy

60

In [2]:
test=datasets.load_from_disk('datasets/gsm8k-test-100.hf')

In [3]:
test['answer']

['Janet sells 16 - 3 - 4 = <<16-3-4=9>>9 duck eggs a day.\nShe makes 9 * 2 = $<<9*2=18>>18 every day at the farmer’s market.\n#### 18',
 'It takes 2/2=<<2/2=1>>1 bolt of white fiber\nSo the total amount of fabric is 2+1=<<2+1=3>>3 bolts of fabric\n#### 3',
 'The cost of the house and repairs came out to 80,000+50,000=$<<80000+50000=130000>>130,000\nHe increased the value of the house by 80,000*1.5=<<80000*1.5=120000>>120,000\nSo the new value of the house is 120,000+80,000=$<<120000+80000=200000>>200,000\nSo he made a profit of 200,000-130,000=$<<200000-130000=70000>>70,000\n#### 70000',
 'He sprints 3*3=<<3*3=9>>9 times\nSo he runs 9*60=<<9*60=540>>540 meters\n#### 540',
 'If each chicken eats 3 cups of feed per day, then for 20 chickens they would need 3*20=<<3*20=60>>60 cups of feed per day.\nIf she feeds the flock 15 cups of feed in the morning, and 25 cups in the afternoon, then the final meal would require 60-15-25=<<60-15-25=20>>20 cups of chicken feed.\n#### 20',
 'The discount

In [42]:
with open('gsm8k_original_responses.pkl', 'rb') as f:
    res=pkl.load(f)

In [43]:
res

["[INST]Answer the following question with just the final number. For example: Question: What is 2+2? \n ### Solution: 4.[/INST] Question: Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?\n### Solution:  The answer to the question is 20",
 '[INST]Answer the following question with just the final number. For example: Question: What is 2+2? \n ### Solution: 4.[/INST] Question: A robe takes 2 bolts of blue fiber and half that much white fiber.  How many bolts in total does it take?\n### Solution:  The robe takes 2 bolts of',
 '[INST]Answer the following question with just the final number. For example: Question: What is 2+2? \n ### Solution: 4.[/INST] Question: Josh decides to try flipping a house.  He buys a house for $80,000 and then puts in $50,000 in repair

In [44]:
gen=[]
for ans in res:
    first_split=ans.split('?\n### Solution:')[-1].strip()
    gen.append(first_split)
    #print(first_split)

    # try:
    #     second_split=first_split.split('answer is')[-1].strip()
    #     #print(second_split)
    #     gen.append(second_split)
    #     continue
    # except:
    #     try:
    #         third=first_split.split('the question is')[-1].strip()
    #         #print(third)
    #         gen.append(third)
    #     except:
    #         pass
    #gen.append(second_split)
    
    # if 'Answer' in second_split:
    #     third_split=second_split.split(': ')[-1]
    #     gen.append(third_split)
    # else:
    #     gen.append(second_split)

In [51]:
len(gen)

100

In [55]:
numbers=[]

for item in gen:
    temp=[]
    x=item.split()
    for i in x:
        if i.isnumeric():
            temp.append(i)
    
    if not temp:
        numbers.extend(' ')
    elif len(temp)>1:
        numbers.extend(temp[0])
    else:
        numbers.extend(temp)


In [56]:
numbers

['20',
 '2',
 ' ',
 '540',
 '30',
 ' ',
 '120',
 '40',
 '120',
 '5',
 '60',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 '12',
 '6',
 '2',
 '7',
 ' ',
 '8',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 '10',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 '6',
 '9',
 '8',
 '1200',
 '32',
 ' ',
 ' ',
 '4',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 '4',
 ' ',
 ' ',
 '7',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 '2',
 ' ',
 ' ',
 ' ',
 '30',
 '24',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 '20',
 ' ',
 ' ',
 ' ',
 ' ',
 '180',
 '4',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 '600',
 '14',
 ' ',
 '4',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 '96',
 '10',
 ' ']

In [57]:
len(numbers)

100

In [58]:
orig=[]
for item in test['answer']:
    orig.append(item.split('####')[-1].strip())

In [59]:
orig

['18',
 '3',
 '70000',
 '540',
 '20',
 '64',
 '260',
 '160',
 '45',
 '460',
 '366',
 '694',
 '13',
 '18',
 '60',
 '125',
 '230',
 '57500',
 '7',
 '6',
 '15',
 '14',
 '7',
 '8',
 '26',
 '2',
 '243',
 '16',
 '25',
 '104',
 '109',
 '80',
 '35',
 '70',
 '23',
 '9',
 '75',
 '2',
 '10',
 '18',
 '8',
 '200',
 '26',
 '48',
 '20',
 '104',
 '163',
 '800',
 '8',
 '30',
 '294',
 '5',
 '15',
 '40',
 '40',
 '14',
 '3',
 '83',
 '57',
 '187',
 '17',
 '1430',
 '25000',
 '1596',
 '300',
 '36',
 '48',
 '595',
 '36',
 '60',
 '7425',
 '60',
 '221',
 '255',
 '88',
 '60',
 '5',
 '100',
 '6',
 '70',
 '10',
 '17',
 '623',
 '600',
 '15',
 '44',
 '22',
 '9360',
 '8000',
 '24',
 '225',
 '28',
 '4',
 '36',
 '348',
 '40',
 '3',
 '12',
 '5',
 '58']

In [60]:
len(orig)

100

In [61]:
accuracy=0
for ori, ans in zip(orig,numbers):
    if ori==ans:
        accuracy+=1

In [62]:
accuracy

5

In [63]:
with open('gsm8k_finetuned_responses.pkl', 'rb') as f:
    res=pkl.load(f)

In [64]:
res

["'Answer the following question through careful, concise step-by-step reasoning. Question: Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?\n### Solution:  The number of eggs she has left after eating and baking is 16 - 3 - 4 = <<16-3-4=9>>9\nShe sells 9 eggs at $2 each for a total of 9 * 2 = $<<9*2=18>>18\n#### 18\n#### 18\n18\n#### 18\n18\n18\n18\n18\n18\n18\n18\n18\n18\n18\n18\n18\n18\n18\n18\n18\n18\n18\n18\n18\n18\n18\n18\n18\n18\n18\n18\n18\n18\n18\n18\n18\n18\n18\n18\n18\n18\n18\n18\n18\n18\n18\n18\n18\n18\n18\n18\n18\n18\n18\n18\n18\n18\n18\n18\n18\n18\n18\n18\n18\n18\n18\n18\n18\n18\n18\n18\n18\n18\n18\n18\n18\n18\n18\n18\n18\n18\n18\n18\n18\n18\n18\n18\n18\n18\n18\n18\n18\n18\n18\n18\n18\n18\n18\n18\n18\n18\n18\n18\n18\n18\n18\n18\n18\n18\n18\n18\

In [73]:
numbers=[]
for item in res:
    temp=[]
    gen=item.split('####')[-1].strip().split('\n')[0].strip()
    x=gen.split()
    for i in x:
        if i.isnumeric():
            temp.append(i)
            break
    
    if not temp:
        numbers.extend(' ')
    elif len(temp)>1:
        numbers.extend(temp[0])
    else:
        numbers.extend(temp)

In [74]:
len(numbers)

100

In [75]:
accuracy=0
for ori, ans in zip(orig,numbers):
    if ori==ans:
        accuracy+=1

In [76]:
accuracy

39

In [2]:
test=datasets.load_from_disk('datasets/sst-test-100.hf')
train=datasets.load_from_disk('datasets/sst-train-900.hf')

In [3]:
with open('sst2_finetuned_responses.pkl', 'rb') as f:
    res=pkl.load(f)

In [4]:
gen=[]
for ans in res:
    first_split=ans.split('Assistant:')[-1].strip()
    print(first_split)


    if 'positive' in first_split:
        gen.append('Positive')
    elif 'negative' in first_split:
        gen.append('Negative')
    else:
        gen.append('lol')

print(len(gen))

The sentiment of the sentence is positive
The sentence has a negative sentiment.
The sentence expresses a positive sentiment
The sentence expresses a positive sentiment
The sentiment of the sentence is negative
The sentiment of the sentence is positive
The sentence has a negative sentiment.
The sentence has a negative sentiment.
The sentence has a positive sentiment.
The sentiment of the sentence is negative
The sentence has a positive sentiment.
The sentence expresses a negative sentiment
The sentiment of the sentence is negative
The sentiment of the sentence is positive
The sentiment of the sentence is negative
The sentiment of the sentence is positive
The sentiment of the sentence is positive
The sentiment of the sentence is positive
The sentiment of the sentence is negative
The sentiment of the sentence is negative
The sentiment of the sentence is mixed
The sentence has a negative sentiment.
The sentiment of the sentence is negative
The sentiment of the sentence is positive
The sen

In [5]:
gen

['Positive',
 'Negative',
 'Positive',
 'Positive',
 'Negative',
 'Positive',
 'Negative',
 'Negative',
 'Positive',
 'Negative',
 'Positive',
 'Negative',
 'Negative',
 'Positive',
 'Negative',
 'Positive',
 'Positive',
 'Positive',
 'Negative',
 'Negative',
 'lol',
 'Negative',
 'Negative',
 'Positive',
 'Positive',
 'Negative',
 'Negative',
 'Positive',
 'Negative',
 'Negative',
 'Positive',
 'Negative',
 'Positive',
 'Negative',
 'Negative',
 'Negative',
 'Positive',
 'Negative',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Negative',
 'Negative',
 'Negative',
 'Positive',
 'Positive',
 'Negative',
 'Negative',
 'Positive',
 'Positive',
 'Positive',
 'Negative',
 'Positive',
 'Negative',
 'Negative',
 'Negative',
 'Negative',
 'Positive',
 'Negative',
 'Positive',
 'Positive',
 'Negative',
 'Negative',
 'Positive',
 'Positive',
 'Positive',
 'Negative',
 'Negative',
 'Positive',
 'Positive',
 'Positive',
 'Negative',
 'Positive',
 'Negative',
 'Po

In [6]:
test['label']

[1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0]

In [7]:
orig=[]
for label in test['label']:
    if label==0:
        orig.append('Negative')
    if label==1:
        orig.append('Positive')

In [8]:
orig

['Positive',
 'Negative',
 'Positive',
 'Positive',
 'Negative',
 'Positive',
 'Negative',
 'Negative',
 'Positive',
 'Negative',
 'Positive',
 'Negative',
 'Negative',
 'Positive',
 'Negative',
 'Positive',
 'Positive',
 'Positive',
 'Negative',
 'Negative',
 'Negative',
 'Negative',
 'Negative',
 'Positive',
 'Positive',
 'Negative',
 'Negative',
 'Positive',
 'Negative',
 'Negative',
 'Positive',
 'Negative',
 'Positive',
 'Negative',
 'Negative',
 'Negative',
 'Positive',
 'Negative',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Negative',
 'Negative',
 'Negative',
 'Positive',
 'Positive',
 'Negative',
 'Negative',
 'Positive',
 'Positive',
 'Positive',
 'Negative',
 'Positive',
 'Negative',
 'Negative',
 'Negative',
 'Negative',
 'Positive',
 'Negative',
 'Positive',
 'Positive',
 'Negative',
 'Negative',
 'Positive',
 'Positive',
 'Positive',
 'Negative',
 'Negative',
 'Positive',
 'Positive',
 'Positive',
 'Negative',
 'Positive',
 'Negative',

In [9]:
accuracy=0
for ori, ans in zip(orig,gen):
    if ori==ans:
        accuracy+=1

In [10]:
accuracy

97

In [2]:
test=datasets.load_from_disk('datasets/qqp-test-100.hf')
train=datasets.load_from_disk('datasets/qqp-train-900.hf')

In [11]:
with open('qqp_finetuned_responses.pkl', 'rb') as f:
    res=pkl.load(f)

In [12]:
res

['[INST]### Human: Are the following sentences semantically equivalent or not? Answer with Yes or No. Sentence 1: Why are African-Americans so beautiful? Sentence 2: Why are hispanics so beautiful?[/INST]\n### Assistant:  The sentences are not sem',
 '[INST]### Human: Are the following sentences semantically equivalent or not? Answer with Yes or No. Sentence 1: I want to pursue PhD in Computer Science about social network,what is the open problem in social networks? Sentence 2: I handle social media for a non-profit. Should I start going to social media networking events? Are there any good ones in the bay area?[/INST]\n### Assistant:  The sentences are not sem',
 '[INST]### Human: Are the following sentences semantically equivalent or not? Answer with Yes or No. Sentence 1: Is there a reason why we should travel alone? Sentence 2: What are some reasons to travel alone?[/INST]\n### Assistant:  The two sentences are not',
 '[INST]### Human: Are the following sentences semantically equiv

In [13]:
gen=[]
for ans in res:
    first_split=ans.split('Assistant:')[-1].strip()
    print(first_split)


    if 'are not' in first_split:
        gen.append('No')
    elif 'are' in first_split:
        gen.append('Yes')
    else:
        gen.append('No')

print(len(gen))

The sentences are not sem
The sentences are not sem
The two sentences are not
The sentences are not sem
The two sentences are not
The two sentences are sem
The two sentences are not
The two sentences are not
The two sentences are sem
The sentences are not sem
The two sentences are not
The sentences are not sem
The two sentences are sem
The sentences are not sem
The sentences are not sem
The answer is Yes.
The two sentences are sem
The sentences are not sem
The two sentences are not
The two sentences are sem
The sentences are not sem
The two sentences are not
The sentences are not sem
The two sentences are not
The sentences are not sem
The sentences are not sem
The two sentences are not
The two sentences are not
The two sentences are not
The sentences are not sem
The two sentences are not
The two sentences are not
The sentences are not sem
The two sentences are not
The two sentences are not
The sentences are not sem
The two sentences are not
The two sentences are not
The two sentences a

In [14]:
orig=[]
for label in test['label']:
    if label==0:
        orig.append('No')
    if label==1:
        orig.append('Yes')

In [15]:
orig

['No',
 'No',
 'Yes',
 'No',
 'No',
 'Yes',
 'Yes',
 'Yes',
 'No',
 'No',
 'No',
 'No',
 'Yes',
 'No',
 'No',
 'Yes',
 'Yes',
 'No',
 'No',
 'Yes',
 'No',
 'Yes',
 'No',
 'No',
 'No',
 'No',
 'No',
 'No',
 'No',
 'No',
 'Yes',
 'No',
 'No',
 'Yes',
 'No',
 'No',
 'No',
 'No',
 'Yes',
 'No',
 'No',
 'No',
 'Yes',
 'No',
 'No',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'No',
 'Yes',
 'No',
 'No',
 'Yes',
 'No',
 'No',
 'No',
 'No',
 'Yes',
 'No',
 'Yes',
 'No',
 'No',
 'No',
 'No',
 'No',
 'No',
 'No',
 'No',
 'Yes',
 'No',
 'No',
 'No',
 'No',
 'No',
 'No',
 'No',
 'No',
 'No',
 'No',
 'Yes',
 'No',
 'Yes',
 'No',
 'Yes',
 'No',
 'No',
 'No',
 'No',
 'No',
 'No',
 'Yes',
 'Yes',
 'Yes',
 'No',
 'No',
 'No',
 'Yes',
 'No']

In [16]:
accuracy=0
for ori, ans in zip(orig,gen):
    if ori==ans:
        accuracy+=1

In [17]:
accuracy

82

In [35]:
test=datasets.load_from_disk('datasets/hellaswag-test-100.hf/')

In [2]:
with open('hs_original_responses.pkl', 'rb') as f:
    res=pkl.load(f)

In [3]:
res

['[INST]### Human: Which of the 4 candidate sentences best completes the initial sentence. Initial Sentence: A man is sitting on a roof. he. Candidate Sentence 0: is using wrap to wrap a pair of skis. Candidate Sentence 1: is ripping level tiles off. Candidate Sentence 2: is holding a rubik\'s cube. Candidate Sentence 3: starts pulling up roofing on a roof.. Answer with just the Sentence number. Some output formats include: ###Assistant 1, ###Assistant 3, ###Assistant 2, ###Assistant 0 [/INST] ###Assistant  The best completion for the initial sentence "A man is sitting on a roof. he" is:\n\n###Assistant 3:',
 '[INST]### Human: Which of the 4 candidate sentences best completes the initial sentence. Initial Sentence: A lady walks to a barbell. She bends down and grabs the pole. the lady. Candidate Sentence 0: swings and lands in her arms. Candidate Sentence 1: pulls the barbell forward. Candidate Sentence 2: pulls a rope attached to the barbell. Candidate Sentence 3: stands and lifts the

In [32]:
gen=[]
for item in res:
    first_split=item.split('The best completion')
    if len(first_split)==2:
        temp=first_split[-1].strip()
        x=temp.split()
        for i in x:
            i=i.split(':')[0]
            if i.isnumeric():
                gen.append(i)
    else:
        second=item.split('The best')[-1].strip()
        x=temp.split()
        for i in x:
            i=i.split(':')[0]
            if i.isnumeric():
                gen.append(i)
        
    

In [33]:
gen

['3',
 '3',
 '2',
 '2',
 '1',
 '2',
 '2',
 '3',
 '1',
 '1',
 '3',
 '3',
 '2',
 '2',
 '1',
 '3',
 '1',
 '0',
 '1',
 '1',
 '0',
 '3',
 '2',
 '2',
 '2',
 '0',
 '1',
 '1',
 '3',
 '1',
 '0',
 '3',
 '2',
 '1',
 '0',
 '2',
 '3',
 '1',
 '2',
 '2',
 '3',
 '2',
 '2',
 '0',
 '1',
 '1',
 '2',
 '2',
 '2',
 '2',
 '1',
 '3',
 '1',
 '3',
 '2',
 '1',
 '3',
 '2',
 '2',
 '1',
 '2',
 '1',
 '2',
 '2',
 '0',
 '1',
 '1',
 '1',
 '2',
 '2',
 '2',
 '2',
 '3',
 '1',
 '1',
 '1',
 '3',
 '1',
 '2',
 '1',
 '1',
 '2',
 '1',
 '2',
 '2',
 '2',
 '1',
 '1',
 '2',
 '1',
 '2',
 '2',
 '1',
 '1',
 '2']

In [37]:
orig=test['label']

In [38]:
accuracy=0
for ori, ans in zip(orig,gen):
    if ori==ans:
        accuracy+=1

In [39]:
accuracy

36

In [40]:
with open('hs_finetuned_responses.pkl', 'rb') as f:
    res=pkl.load(f)

In [41]:
res

["[INST]### Human: Which of the 4 candidate sentences best completes the initial sentence. Initial Sentence: A man is sitting on a roof. he. Candidate Sentence 0: is using wrap to wrap a pair of skis. Candidate Sentence 1: is ripping level tiles off. Candidate Sentence 2: is holding a rubik's cube. Candidate Sentence 3: starts pulling up roofing on a roof.. Answer with just the Sentence number. Some output formats include: ###Assistant 1, ###Assistant 3, ###Assistant 2, ###Assistant 0 [/INST] ###Assistant  The best completion for the given prompt is:\n\n2. The",
 '[INST]### Human: Which of the 4 candidate sentences best completes the initial sentence. Initial Sentence: A lady walks to a barbell. She bends down and grabs the pole. the lady. Candidate Sentence 0: swings and lands in her arms. Candidate Sentence 1: pulls the barbell forward. Candidate Sentence 2: pulls a rope attached to the barbell. Candidate Sentence 3: stands and lifts the weight over her head.. Answer with just the Se

In [42]:
gen=[]
for item in res:    
    first_split=item.split('The best completion')
    if len(first_split)==2:
        temp=first_split[-1].strip()
        x=temp.split()
        for i in x:
            i=i.split(':')[0]
            if i.isnumeric():
                gen.append(i)
                break
    
    else:
        second=item.split('The correct')[-1].strip()
        x=temp.split()
        for i in x:
            i=i.split(':')[0]
            if i.isnumeric():
                gen.append(i)

In [43]:
gen

['2', '2', '2']

In [3]:
test=datasets.load_from_disk('datasets/cmsqa-test-100.hf/')

In [7]:
test['answerKey']

['A',
 'A',
 'B',
 'A',
 'A',
 'C',
 'B',
 'D',
 'A',
 'C',
 'E',
 'D',
 'A',
 'D',
 'C',
 'D',
 'D',
 'E',
 'E',
 'D',
 'D',
 'C',
 'D',
 'D',
 'E',
 'D',
 'B',
 'A',
 'B',
 'B',
 'B',
 'A',
 'E',
 'D',
 'E',
 'C',
 'C',
 'B',
 'A',
 'B',
 'B',
 'B',
 'E',
 'E',
 'D',
 'E',
 'A',
 'E',
 'E',
 'C',
 'C',
 'B',
 'D',
 'C',
 'D',
 'E',
 'B',
 'D',
 'B',
 'A',
 'B',
 'E',
 'C',
 'D',
 'A',
 'A',
 'D',
 'A',
 'C',
 'D',
 'E',
 'E',
 'D',
 'A',
 'C',
 'B',
 'C',
 'E',
 'B',
 'C',
 'B',
 'D',
 'E',
 'A',
 'E',
 'A',
 'D',
 'B',
 'E',
 'C',
 'C',
 'C',
 'D',
 'E',
 'E',
 'E',
 'B',
 'D',
 'B',
 'E']

In [4]:
with open('cmsqa_original_responses.pkl', 'rb') as f:
    res=pkl.load(f)

In [5]:
res

['[INST]### Human: Which one of the 5 choices best answers the question. Question: A revolving door is convenient for two direction travel, but it also serves as a security measure at a what?. Choice A: bank, Choice B: library, Choice C: department store, Choice D: mall, Choice E: new york. [/INST]\n### Assistant:  The correct answer is Choice A: bank. A revolving door is often used in banks as a security measure to control access and prevent una',
 '[INST]### Human: Which one of the 5 choices best answers the question. Question: What do people aim to do at work?. Choice A: complete job, Choice B: learn from each other, Choice C: kill animals, Choice D: wear hats, Choice E: talk to each other. [/INST]\n### Assistant:  The best answer to the question "What do people aim to do at work?" is Choice A: complete job. This is because the primary',
 '[INST]### Human: Which one of the 5 choices best answers the question. Question: Where would you find magazines along side many other printed wor

In [12]:
gen=[]
for item in res:    
    first_split=item.split('### Assistant:')[-1].strip()
    #print(first_split)
    if 'A' in first_split:
        gen.append('A')
    elif 'B' in first_split:
        gen.append('B')
    elif 'C' in first_split:
        gen.append('C')
    elif 'D' in first_split:
        gen.append('D')
    elif 'E' in first_split:
        gen.append('E')
    else:
        gen.append('X')

In [13]:
gen

['A',
 'A',
 'A',
 'A',
 'X',
 'B',
 'A',
 'E',
 'B',
 'C',
 'B',
 'D',
 'A',
 'A',
 'C',
 'D',
 'B',
 'A',
 'E',
 'C',
 'A',
 'C',
 'D',
 'C',
 'X',
 'D',
 'B',
 'A',
 'A',
 'B',
 'D',
 'C',
 'C',
 'C',
 'C',
 'C',
 'A',
 'B',
 'A',
 'A',
 'B',
 'A',
 'B',
 'B',
 'D',
 'A',
 'A',
 'B',
 'A',
 'A',
 'C',
 'X',
 'C',
 'C',
 'D',
 'E',
 'B',
 'C',
 'C',
 'E',
 'B',
 'C',
 'C',
 'C',
 'A',
 'A',
 'D',
 'A',
 'C',
 'C',
 'E',
 'D',
 'D',
 'D',
 'C',
 'B',
 'C',
 'C',
 'B',
 'X',
 'B',
 'A',
 'E',
 'A',
 'C',
 'B',
 'D',
 'B',
 'A',
 'C',
 'X',
 'A',
 'A',
 'C',
 'A',
 'C',
 'B',
 'D',
 'B',
 'X']

In [14]:
orig=test['answerKey']

In [15]:
accuracy=0
for ori, ans in zip(orig,gen):
    if ori==ans:
        accuracy+=1

In [17]:
accuracy

48

In [24]:
with open('cmsqa_finetuned_responses.pkl', 'rb') as f:
    res=pkl.load(f)

In [25]:
res

['[INST]### Human: Which one of the 5 choices best answers the question. Question: A revolving door is convenient for two direction travel, but it also serves as a security measure at a what?. Choice A: bank, Choice B: library, Choice C: department store, Choice D: mall, Choice E: new york. [/INST]\n### Assistant:  The correct answer is A: bank. A',
 '[INST]### Human: Which one of the 5 choices best answers the question. Question: What do people aim to do at work?. Choice A: complete job, Choice B: learn from each other, Choice C: kill animals, Choice D: wear hats, Choice E: talk to each other. [/INST]\n### Assistant:  The correct answer is A: complete job.',
 '[INST]### Human: Which one of the 5 choices best answers the question. Question: Where would you find magazines along side many other printed works?. Choice A: doctor, Choice B: bookstore, Choice C: market, Choice D: train station, Choice E: mortuary. [/INST]\n### Assistant:  The correct answer is B: bookstore.',
 '[INST]### Hum

In [26]:
gen=[]
for item in res:    
    first_split=item.split('### Assistant:')[-1].strip()
    #print(first_split)
    if 'A' in first_split:
        gen.append('A')
    elif 'B' in first_split:
        gen.append('B')
    elif 'C' in first_split:
        gen.append('C')
    elif 'D' in first_split:
        gen.append('D')
    elif 'E' in first_split:
        gen.append('E')
    else:
        gen.append('X')

In [27]:
gen

['A',
 'A',
 'B',
 'A',
 'A',
 'B',
 'B',
 'E',
 'A',
 'C',
 'B',
 'D',
 'A',
 'D',
 'C',
 'D',
 'D',
 'E',
 'E',
 'A',
 'D',
 'C',
 'E',
 'D',
 'E',
 'D',
 'B',
 'A',
 'B',
 'B',
 'D',
 'C',
 'E',
 'D',
 'E',
 'C',
 'A',
 'B',
 'A',
 'C',
 'B',
 'B',
 'E',
 'E',
 'D',
 'E',
 'B',
 'E',
 'D',
 'C',
 'C',
 'B',
 'D',
 'C',
 'D',
 'E',
 'B',
 'D',
 'C',
 'B',
 'B',
 'E',
 'C',
 'D',
 'A',
 'A',
 'D',
 'D',
 'C',
 'D',
 'E',
 'E',
 'X',
 'A',
 'C',
 'B',
 'C',
 'E',
 'B',
 'C',
 'B',
 'D',
 'E',
 'A',
 'E',
 'A',
 'D',
 'A',
 'A',
 'C',
 'E',
 'C',
 'D',
 'E',
 'E',
 'C',
 'B',
 'D',
 'B',
 'E']

In [28]:
accuracy=0
for ori, ans in zip(orig,gen):
    if ori==ans:
        accuracy+=1

In [29]:
accuracy

81

In [30]:
test=datasets.load_from_disk('datasets/mmlu-test-100.hf/')

In [31]:
with open('mmlu_original_responses.pkl', 'rb') as f:
    res=pkl.load(f)

In [33]:
gen=[]
for item in res:    
    first_split=item.split('### Assistant:')[-1].strip()
    #print(first_split)
    if '0' in first_split:
        gen.append('0')
    elif '1' in first_split:
        gen.append('1')
    elif '2' in first_split:
        gen.append('2')
    elif '3' in first_split:
        gen.append('3')
    else:
        gen.append('X')

In [34]:
gen

['0',
 '0',
 '2',
 '3',
 '1',
 '2',
 '0',
 '0',
 '3',
 'X',
 '1',
 '0',
 '1',
 '0',
 '1',
 '2',
 '2',
 '2',
 '3',
 '3',
 '1',
 '2',
 '1',
 '1',
 '1',
 '3',
 '1',
 '3',
 '2',
 '2',
 '1',
 '1',
 'X',
 '2',
 '2',
 '1',
 '1',
 '2',
 '1',
 '2',
 '1',
 '2',
 '1',
 '0',
 '2',
 '0',
 '2',
 '2',
 '1',
 '3',
 '2',
 '1',
 '3',
 '2',
 '0',
 '2',
 '0',
 '1',
 '0',
 '3',
 '2',
 '2',
 '0',
 '2',
 '1',
 '2',
 '2',
 '0',
 '1',
 '0',
 '1',
 'X',
 '2',
 '1',
 '1',
 '1',
 '2',
 '1',
 '3',
 '1',
 '0',
 '0',
 'X',
 'X',
 'X',
 '1',
 '2',
 '1',
 '3',
 '1',
 '1',
 '0',
 '0',
 '0',
 '1',
 '0',
 '3',
 '2',
 '1',
 '1',
 '3',
 'X',
 '1',
 '2',
 '1',
 'X',
 '2',
 '2',
 '1',
 '0',
 '2',
 'X',
 '0',
 '1']

In [35]:
orig=test['answer']

In [38]:
orig

[1,
 2,
 0,
 3,
 3,
 3,
 0,
 2,
 3,
 1,
 1,
 0,
 0,
 1,
 0,
 2,
 1,
 2,
 2,
 3,
 1,
 3,
 1,
 1,
 1,
 0,
 1,
 0,
 2,
 0,
 3,
 1,
 0,
 3,
 1,
 0,
 3,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 0,
 0,
 2,
 0,
 3,
 0,
 1,
 1,
 3,
 1,
 0,
 3,
 0,
 0,
 3,
 3,
 1,
 0,
 3,
 3,
 0,
 2,
 2,
 2,
 1,
 0,
 1,
 2,
 3,
 0,
 0,
 1,
 2,
 3,
 3,
 1,
 2,
 2,
 3,
 1,
 2,
 3,
 0,
 1,
 3,
 3,
 1,
 0,
 0,
 1,
 2,
 0,
 3,
 2,
 1,
 0,
 3,
 3,
 0,
 2,
 1,
 0,
 2,
 1,
 1,
 1,
 2,
 0,
 2,
 1]

In [39]:
accuracy=0
for ori, ans in zip(orig,gen):
    if str(ori)==ans:
        accuracy+=1

In [40]:
accuracy

51

In [45]:
with open('mmlu_finetuned_responses.pkl', 'rb') as f:
    res=pkl.load(f)

In [46]:
gen=[]
for item in res:    
    first_split=item.split('### Assistant:')[-1].strip()
    #print(first_split)
    if '0' in first_split:
        gen.append('0')
    elif '1' in first_split:
        gen.append('1')
    elif '2' in first_split:
        gen.append('2')
    elif '3' in first_split:
        gen.append('3')
    else:
        gen.append('X')

In [47]:
accuracy=0
for ori, ans in zip(orig,gen):
    if str(ori)==ans:
        accuracy+=1

In [48]:
accuracy

61

In [2]:
test=datasets.load_from_disk('datasets/banking77-test-100.hf/')

In [3]:
with open('banking77_finetuned_responses.pkl', 'rb') as f:
    res=pkl.load(f)

In [4]:
choices=list(set(test['label_text']))

In [5]:
gen=[]
for item in res:    
    first_split=item.split('### Assistant:')[-1].strip()
    print(first_split)
    gen.append(first_split)
    # if '0' in first_split:
    #     gen.append('0')
    # elif '1' in first_split:
    #     gen.append('1')
    # elif '2' in first_split:
    #     gen.append('2')
    # elif '3' in first_split:
    #     gen.append('3')
    # else:
    #     gen.append('X')

The_top_up_by_cash_not_working
The_top_up_limits_have_been_re
The_cash_top_up_limits_are_d
The_top_up_limits_have_been_re
The_cash_withdrawal_not_working_top
The_cash_withdrawal_not_working_c
The_end_of_the_line_for_my_
The_account_closure_charge_reimbursement_charge
The_transfer_of_funds_into_my_
The_exchange_rate_for_cash_withdrawal
The_charge_for_transferring_money_into
The_cost_of_transferring_money_via
The_card_is_not_working_for_contactless
The_card_is_not_working_for_contactless
The_card_is_not_working_and_I_
The_card_arrival_estimate_has_passed
The_Holding_Currency_Supported_Currencies
The_exchange_office_support_exchange_by_cash
The_payment_was_declined_or_not_accept
The_payment_has_not_been_received_
The_beneficiary_not_receiving_funds
The_beneficiary_account_not_allowed_for
The_charge_was_not_recognised_by_my
The_charge_was_not_recognised_by_the
The_transfer_not_received_by_recip
The_transfer_has_not_arrived_y
The_card_acceptance_limits_are_different_
The_card_acceptance_lim

In [6]:
orig=test['label_text']

In [7]:
orig

['apple_pay_or_google_pay',
 'apple_pay_or_google_pay',
 'supported_cards_and_currencies',
 'supported_cards_and_currencies',
 'pending_cash_withdrawal',
 'pending_cash_withdrawal',
 'terminate_account',
 'terminate_account',
 'receiving_money',
 'receiving_money',
 'top_up_by_bank_transfer_charge',
 'top_up_by_bank_transfer_charge',
 'contactless_not_working',
 'contactless_not_working',
 'card_arrival',
 'card_arrival',
 'fiat_currency_support',
 'fiat_currency_support',
 'pending_card_payment',
 'pending_card_payment',
 'beneficiary_not_allowed',
 'beneficiary_not_allowed',
 'transfer_fee_charged',
 'transfer_fee_charged',
 'transfer_not_received_by_recipient',
 'transfer_not_received_by_recipient',
 'atm_support',
 'atm_support',
 'getting_spare_card',
 'getting_spare_card',
 'country_support',
 'country_support',
 'verify_source_of_funds',
 'verify_source_of_funds',
 'pending_transfer',
 'pending_transfer',
 'edit_personal_details',
 'edit_personal_details',
 'get_disposable_virtu

In [8]:
accuracy=0
for ori, ans in zip(orig,gen):
    if str(ori)==ans:
        accuracy+=1

In [9]:
accuracy

0

In [10]:
gen

['The_top_up_by_cash_not_working',
 'The_top_up_limits_have_been_re',
 'The_cash_top_up_limits_are_d',
 'The_top_up_limits_have_been_re',
 'The_cash_withdrawal_not_working_top',
 'The_cash_withdrawal_not_working_c',
 'The_end_of_the_line_for_my_',
 'The_account_closure_charge_reimbursement_charge',
 'The_transfer_of_funds_into_my_',
 'The_exchange_rate_for_cash_withdrawal',
 'The_charge_for_transferring_money_into',
 'The_cost_of_transferring_money_via',
 'The_card_is_not_working_for_contactless',
 'The_card_is_not_working_for_contactless',
 'The_card_is_not_working_and_I_',
 'The_card_arrival_estimate_has_passed',
 'The_Holding_Currency_Supported_Currencies',
 'The_exchange_office_support_exchange_by_cash',
 'The_payment_was_declined_or_not_accept',
 'The_payment_has_not_been_received_',
 'The_beneficiary_not_receiving_funds',
 'The_beneficiary_account_not_allowed_for',
 'The_charge_was_not_recognised_by_my',
 'The_charge_was_not_recognised_by_the',
 'The_transfer_not_received_by_reci

In [27]:
test=datasets.load_from_disk('datasets/isarcasm-test-100.hf/')

In [60]:
with open('sarcasm_finetuned_responses.pkl', 'rb') as f:
    res=pkl.load(f)

In [61]:
gen=[]
for item in res:    
    first_split=item.split('### Assistant:')[-1].strip()
    #print(first_split)
    if "Yes" in first_split or "is sarcastic" in first_split or "contains" in first_split or "interpreted as sarcastic" in first_split:
        gen.append('1')
    elif "No" in first_split or "is not sarcastic" in first_split or "does not contain" in first_split or 'is not' in first_split:
        gen.append('0')
    else:
        gen.append('X')
        print(first_split)
    # gen.append(first_split)

The sentence "get a jab couldn't be easy, go
The sentence "Let's keep building the Africa that we want"
The sentence "down to 10 men hold on ox" is
The sentence "pleasure be all ours, thank you" is
The sentence "i live for the great british weather" is sar


In [62]:
gen

['1',
 '0',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '0',
 '1',
 '1',
 '1',
 '1',
 '1',
 '0',
 '0',
 '1',
 '1',
 '0',
 '0',
 '1',
 '1',
 '0',
 '0',
 '1',
 'X',
 '1',
 '0',
 '1',
 '1',
 '1',
 '1',
 '0',
 '1',
 '0',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 'X',
 '1',
 '1',
 '0',
 '1',
 '0',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '0',
 '1',
 '1',
 '0',
 '1',
 '0',
 '0',
 '0',
 '1',
 '1',
 '1',
 'X',
 '1',
 'X',
 '1',
 '1',
 '1',
 '0',
 '1',
 '1',
 '1',
 '0',
 '0',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 'X',
 '0',
 '0',
 '0',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '0']

In [63]:
orig=test['label']

In [64]:
accuracy=0
skip=0
for ori, ans in zip(orig,gen):
    if ans=='X':
        skip+=1
    if str(ori)==ans:
        accuracy+=1

In [65]:
accuracy

68

In [66]:
skip

5